In [1]:
import pandas as pd
import numpy as np
import geocoder

In [2]:
# data = pd.read_csv("/home/shin/Dropbox/CV/Accepted/20180822_Optiopay/test/OptioPay Analytics Challenge_Dataset.csv", delimiter=";")

In [97]:
data = pd.read_csv("/media/shin/OS/Documents and Settings/shinj/Dropbox/CV/Accepted/20180822_Optiopay/test/OptioPay Analytics Challenge_Dataset.csv", delimiter=";")

In [98]:
city_list = data.first_city.append(data.last_city).unique()
city_list = city_list[~pd.isnull(city_list)].tolist()

In [99]:
g = geocoder.bing('Hamburg', key='**')

Status code 401 from http://dev.virtualearth.net/REST/v1/Locations: ERROR - 401 Client Error: Unauthorized for url: http://dev.virtualearth.net/REST/v1/Locations?q=Hamburg&o=json&inclnb=1&key=%2A%2A&maxResults=1


In [100]:
import csv

In [101]:
with open('geodata.csv','w') as f:
    w = csv.writer(f)
    w.writerow(geodata.items())

In [102]:
data_geo = data.drop(['PJ_ID', 'pj_created_at', 'recipient_details_gender', 'mpj',
       'first_device', 'first_os', 'first_browser', 'campaign_viewed', 'outcome',
        'num_campaign_pages_looked', 'num_campaigns_looked', 'Closed',
       'last_device', 'summarized_last_device', 'last_os', 'last_browser',
        'video_viewed', 'site visit', 'PJ count'], axis = 1)

In [46]:
first_city = data_geo.drop(['last_city'], axis = 1)
first_city = first_city.dropna()
first_city = first_city[first_city['conversion'] > 0]

In [78]:
fc_plot = first_city.groupby(['first_city']).sum()

In [79]:
fc_plot = fc_plot.sort_values(['conversion'], ascending= False)[:20]

In [80]:
geodata = {}
for i in fc_plot.index:
    try:
        g = geocoder.bing(i, key='AvN8qlYVCn9vE_rDZSL9tH_B-NPPqfN7kTK1-HTRuk5lPzD_Bd9ATrz74VCNPSzG')
        geodata.update({i:g.latlng})
    except:
        pass

In [81]:
fc_lat = []
fc_lng = []
for i in fc_plot.index:
    if i in geodata:
        fc_lat.append(geodata.get(i)[0])
        fc_lng.append(geodata.get(i)[1])
    else:
        fc_lat.append(0)
        fc_lng.append(0)

In [82]:
fc_plot['lat'] = fc_lat
fc_plot['lon'] = fc_lng

In [83]:
fc_plot

,conversion,lat,lon
first_city,,,
Berlin,43,52.501404,13.402324
Hamburg,32,53.553341,9.992470
Stuttgart,12,48.767761,9.171990
Wuppertal,12,51.271599,7.196680
Munich,12,48.136410,11.577540
Dusseldorf,12,51.215630,6.776040
Cologne,11,50.941669,6.955160
Frankfurt am Main,10,50.112080,8.683410
Dortmund,9,51.516609,7.458290


In [84]:
df = fc_plot

In [85]:
df

,conversion,lat,lon
first_city,,,
Berlin,43,52.501404,13.402324
Hamburg,32,53.553341,9.992470
Stuttgart,12,48.767761,9.171990
Wuppertal,12,51.271599,7.196680
Munich,12,48.136410,11.577540
Dusseldorf,12,51.215630,6.776040
Cologne,11,50.941669,6.955160
Frankfurt am Main,10,50.112080,8.683410
Dortmund,9,51.516609,7.458290


In [86]:
import plotly.plotly as py
import plotly.graph_objs as go

In [90]:
df['text'] = df.index + ' Conversion ' + (df['conversion']).astype(str)

scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = [ dict(
        type = 'scattergeo',
        locationmode = 'country names',
        lon = df['lon'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        marker = dict(
            size = 12,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'round',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = df['conversion'],
            cmax = df['conversion'].max(),
            colorbar=dict(
                title="Top 20 Cities of Conversion"
            )
        ))]

layout = dict(
        title = 'Top 20 Cities of Conversion in Germany',
        colorbar = True,
        geo = dict(
            scope='europe',
            projection=dict( type='mecator' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-airports' )

In [111]:
tr_city = data_geo.drop(['last_city'], axis = 1)
tr_city = tr_city.dropna()
tr_city = tr_city.groupby(['first_city']).count().sort_values(['conversion'], ascending = False)[:20]

In [113]:
geodata = {}
for i in tr_city.index:
    try:
        g = geocoder.bing(i, key='AvN8qlYVCn9vE_rDZSL9tH_B-NPPqfN7kTK1-HTRuk5lPzD_Bd9ATrz74VCNPSzG')
        geodata.update({i:g.latlng})
    except:
        pass

In [115]:
tr_lat = []
tr_lng = []
for i in tr_city.index:
    if i in geodata:
        tr_lat.append(geodata.get(i)[0])
        tr_lng.append(geodata.get(i)[1])
    else:
        tr_lat.append(0)
        tr_lng.append(0)

In [117]:
tr_city['lat'] = tr_lat
tr_city['lon'] = tr_lng

In [118]:
df = tr_city

In [121]:
df['text'] = df.index + ' Conversion ' + (df['conversion']).astype(str)

scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

data = [ dict(
        type = 'scattergeo',
        locationmode = 'country names',
        lon = df['lon'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        marker = dict(
            size = 12,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'round',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = df['conversion'],
            cmax = df['conversion'].max(),
            colorbar=dict(
                title="Top 20 Cities of Transaction"
            )
        ))]

layout = dict(
        title = 'Top 20 Cities of Transaction in Germany',
        colorbar = True,
        geo = dict(
            scope='europe',
            projection=dict( type='mecator' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-airports' )

In [123]:
tr_city.columns = ['Transaction', 'lat', 'lon', 'text']

In [124]:
tr_city

,Transaction,lat,lon,text
first_city,,,,
Berlin,729,52.501404,13.402324,Berlin Conversion 729
Hamburg,544,53.553341,9.992470,Hamburg Conversion 544
Wuppertal,251,51.271599,7.196680,Wuppertal Conversion 251
Cologne,214,50.941669,6.955160,Cologne Conversion 214
Munich,192,48.136410,11.577540,Munich Conversion 192
Frankfurt am Main,189,50.112080,8.683410,Frankfurt am Main Conversion 189
Hanover,182,52.372269,9.738150,Hanover Conversion 182
Stuttgart,176,48.767761,9.171990,Stuttgart Conversion 176
Dusseldorf,173,51.215630,6.776040,Dusseldorf Conversion 173
